In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import pandas as pd

In [3]:
import json

In [4]:
import pandas as pd

events_df = pd.read_csv('anchors.csv')

from beads.simple import SimpleExecutor

from beads.composed import ComposedExecutor

# Процессинг старых ивентов

In [5]:
def time_to_seconds(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    return hours * 3600 + minutes * 60 + seconds

events_df['Start_time'] = events_df['Start'].apply(lambda x: time_to_seconds(x))
events_df['End_time'] = events_df['End'].apply(lambda x: time_to_seconds(x))

query_events_derivative_df = events_df[events_df['Type'] == 'derivative'].rename(columns={
    'Participant': 'user_id',
    'Feature': 'type',
    'Start_time': 'time',
})[['user_id', 'type', 'time']]

query_events_non_derivative_starts_df = events_df[events_df['Type'] == 'non-derivative'].rename(columns={
    'Participant': 'user_id',
    'Feature': 'type',
    'Start_time': 'time',
})[['user_id', 'type', 'time']]

query_events_non_derivative_starts_df['type'] += '_start'

query_events_non_derivative_starts_df['start_id'] = query_events_non_derivative_starts_df.index

query_events_non_derivative_ends_df = events_df[events_df['Type'] == 'non-derivative'].rename(columns={
    'Participant': 'user_id',
    'Feature': 'type',
    'End_time': 'time',
})[['user_id', 'type', 'time']]

query_events_non_derivative_ends_df['end_id'] = query_events_non_derivative_starts_df.index

query_events_non_derivative_ends_df['type'] += '_end'

query_events_df = pd.concat([
    query_events_derivative_df,
    query_events_non_derivative_starts_df,
    query_events_non_derivative_ends_df
]).reset_index(drop=True)

# query_events_df

query_events_df['session_id'] = query_events_df['user_id']

composed = ComposedExecutor(query_events_df)

In [6]:
# composed.query_events_df

In [7]:
simple_executor = SimpleExecutor(composed.query_events_df)

# Что дальше?

Для Not, возможно, есть ошибка - 

In [169]:
query = [{
        'node_conditions': ['type == "round_3_start"'],
    }, {
        'node_conditions': ['type == "camp_combat_start"']
    }, {
        'node_conditions': ['type == "camp_combat_end"'],
        'order_conditions': [{
            'node_used': 1,
            'condition': 'start_id_A == end_id_B'
        }]
    }, {
        'not': [{
            'node_conditions': ['type == "camp_combat_start"']
        }]
    }, {
        'node_conditions': ['type == "round_3_end"'],
        'order_conditions': [{
            'node_used': 0,
            'condition': 'start_id_A == end_id_B'
        }]
    }
]

Re

In [170]:
def index_query(query, base_index):
    if isinstance(query, list):
        return [index_query(e, f'{base_index}_{i}') for i, e in enumerate(query)]

    elif isinstance(query, dict) and any(key in query for key in ['not', 'and', 'or']):
        operator = list(query.keys())[0]  # Получаем оператор ('not', 'and', 'or')
        return (f'{base_index}_{operator}', {
            operator: [
                index_query(arg, f'{base_index}_{operator}_{i}') 
                for i, arg in enumerate(query[operator])
            ]
        })

    else:
        return (base_index, query)

In [171]:
indexed_query = index_query(query, 'e')

In [172]:
def compose_query_logic(query):
    base_query = []
    translated_queries = []
    
    for index, element in query:
        if isinstance(element, dict):  # If the element is a dictionary
            key = next(iter(element))  # Get the 'not', 'and', or 'or' key
            if key == 'not':  # Unary operator
                translated_queries.append({
                    'type': 'not',
                    'query': base_query + element[key]
                })
            elif key in ('and', 'or'):  # 'and' or 'or' n-ary operators
                translated_queries.append({
                    'type': key,
                    'queries': [
                        base_query + sub_element
                        for sub_element in element[key]
                    ]
                })
                    
            else:
                base_query.append((index, element))
        else:  # If the element is not a dictionary, it's a base query component
            base_query += [(index, element)]
    
    # Add the base query at the beginning of the result list
    translated_queries.insert(0, {'type': 'base', 'query': base_query})
    
    return translated_queries

In [187]:
# # Assuming the A0, B, A1, C, D, E, A2, F, G, H are already defined tuples
# # For example:
# A0 = [(0, 'A0_query')]
# B = [(1, 'B_query')]
# A1 = [(2, 'A1_query')]
# C = [(3, 'C_query')]
# D = [(4, 'D_query')]
# E = [(5, 'E_query')]
# A2 = [(6, 'A2_query')]
# F = [(7, 'F_query')]
# G = [(8, 'G_query')]
# H = [(9, 'H_query')]

# # The initial query list
# query = [
#     A0, {
#         'not': B
#     }, A1, {
#         'and': [C, D, E]
#     }, A2, {
#         'or': [F, G, H]
#     }
# ]

# # Function to concatenate tuples from different queries
# def concatenate(queries):
#     return sum(queries, [])

# # The translation function
# def translate_query(query):
#     base_query = []
#     translated_queries = []

#     for index, item in query:
#         if isinstance(item, dict):
#             # Logical operators
#             for key, value in item.items():
#                 if key == 'not':
#                     translated_queries.append({
#                         'type': 'not',
#                         'query': concatenate([base_query, value, base_query])
#                     })
#                 elif key == 'and':
#                     for subquery in value:
#                         translated_queries.append({
#                             'type': 'and',
#                             'query': concatenate([base_query, subquery, base_query])
#                         })
#                 elif key == 'or':
#                     for subquery in value:
#                         translated_queries.append({
#                             'type': 'or',
#                             'query': concatenate([base_query, subquery])
#                         })

#     # Adding the base query at the beginning
#     translated_queries.insert(0, {'type': 'base', 'query': base_query})

#     return translated_queries

# # Translate the query
# queries = translate_query(query)

# # Output the translated queries
# for q in queries:
#     print(q)

{'type': 'base', 'query': [(0, 'A0_query'), (2, 'A1_query'), (6, 'A2_query')]}
{'type': 'not', 'query': [(0, 'A0_query'), (1, 'B_query'), (0, 'A0_query')]}
{'type': 'and', 'query': [(0, 'A0_query'), (2, 'A1_query'), (3, 'C_query'), (0, 'A0_query'), (2, 'A1_query')]}
{'type': 'and', 'query': [(0, 'A0_query'), (2, 'A1_query'), (4, 'D_query'), (0, 'A0_query'), (2, 'A1_query')]}
{'type': 'and', 'query': [(0, 'A0_query'), (2, 'A1_query'), (5, 'E_query'), (0, 'A0_query'), (2, 'A1_query')]}
{'type': 'or', 'query': [(0, 'A0_query'), (2, 'A1_query'), (6, 'A2_query'), (7, 'F_query')]}
{'type': 'or', 'query': [(0, 'A0_query'), (2, 'A1_query'), (6, 'A2_query'), (8, 'G_query')]}
{'type': 'or', 'query': [(0, 'A0_query'), (2, 'A1_query'), (6, 'A2_query'), (9, 'H_query')]}


In [185]:
indexed_query

[('e_0', {'node_conditions': ['type == "round_3_start"']}),
 ('e_1', {'node_conditions': ['type == "camp_combat_start"']}),
 ('e_2',
  {'node_conditions': ['type == "camp_combat_end"'],
   'order_conditions': [{'node_used': 1,
     'condition': 'start_id_A == end_id_B'}]}),
 ('e_3_not',
  {'not': [('e_3_not_0',
     {'node_conditions': ['type == "camp_combat_start"']})]}),
 ('e_4',
  {'node_conditions': ['type == "round_3_end"'],
   'order_conditions': [{'node_used': 0,
     'condition': 'start_id_A == end_id_B'}]})]

In [174]:
compose_query_logic(indexed_query)

[{'type': 'base',
  'query': [('e_0', {'node_conditions': ['type == "round_3_start"']}),
   ('e_1', {'node_conditions': ['type == "camp_combat_start"']}),
   ('e_2',
    {'node_conditions': ['type == "camp_combat_end"'],
     'order_conditions': [{'node_used': 1,
       'condition': 'start_id_A == end_id_B'}]}),
   ('e_4',
    {'node_conditions': ['type == "round_3_end"'],
     'order_conditions': [{'node_used': 0,
       'condition': 'start_id_A == end_id_B'}]})]},
 {'type': 'not',
  'query': [('e_0', {'node_conditions': ['type == "round_3_start"']}),
   ('e_1', {'node_conditions': ['type == "camp_combat_start"']}),
   ('e_2',
    {'node_conditions': ['type == "camp_combat_end"'],
     'order_conditions': [{'node_used': 1,
       'condition': 'start_id_A == end_id_B'}]}),
   ('e_3_not_0', {'node_conditions': ['type == "camp_combat_start"']})]}]

Нужно выполнить запросы один за одним, и затем примержить правильно

In [175]:
def find_intersection(df1, df2):
    return [c for c in df1.columns if c in df2.columns]

In [176]:
def merge_queries(queries):
    base_query = queries[0]['query']
    base_result_df = execute_query(base_query)

    for query in queries[1:]:
        if query['type'] == 'not':
            not_result_df = execute_query(query['query'])
            events_intersection = find_intersection(base_result_df, not_result_df)
            not_result_df['remove'] = True
            base_result_df = base_result_df.merge(not_result_df, on=events_intersection, how='left')
            base_result_df = base_result_df[base_result_df['remove'] != True].drop(columns='remove').copy()

        if query['type'] == 'and':
            for subquery in query['queries']:
                and_result_df = execute_query(subquery)
                events_intersection = find_intersection(base_result_df, and_result_df)
                and_result_df['save'] = True
                base_result_df = base_result_df.merge(and_result_df, on=events_intersection, how='left')
                base_result_df = base_result_df[base_result_df['save'] == True].drop(columns='save').copy()

        if query['type'] == 'or':
            for i, subquery in enumerate(query['queries']):
                or_result_df = execute_query(subquery)
                events_intersection = find_intersection(base_result_df, or_result_df)
                or_result_df[f'save_{i}'] = 1
                base_result_df = base_result_df.merge(or_result_df, on=events_intersection, how='left')

            or_fields = [f'save_{i}' for i, _ in enumerate(query['queries'])]
            base_result_df['save'] = base_result_df[or_fields].sum(axis=1)
            base_result_df = base_result_df[base_result_df['save'] > 0].drop(columns=['save'] + or_fields).copy()
    
    return base_result_df

In [177]:
def is_simple_query(query):
    if not isinstance(query, list):
        assert False, f"Wrong query given! {query}"
    for index, element in query:
        if ('node_conditions' not in element):
            return False
    return True

In [178]:
def execute_query(query):
    if is_simple_query(query):
        return simple_executor.execute(query)
    
    queries = compose_query_logic(query)
    print(queries)
    return merge_queries(queries)

In [180]:
is_simple_query(indexed_query)

False

In [181]:
a_df = execute_query(indexed_query)

[{'type': 'base', 'query': [('e_0', {'node_conditions': ['type == "round_3_start"']}), ('e_1', {'node_conditions': ['type == "camp_combat_start"']}), ('e_2', {'node_conditions': ['type == "camp_combat_end"'], 'order_conditions': [{'node_used': 1, 'condition': 'start_id_A == end_id_B'}]}), ('e_4', {'node_conditions': ['type == "round_3_end"'], 'order_conditions': [{'node_used': 0, 'condition': 'start_id_A == end_id_B'}]})]}, {'type': 'not', 'query': [('e_0', {'node_conditions': ['type == "round_3_start"']}), ('e_1', {'node_conditions': ['type == "camp_combat_start"']}), ('e_2', {'node_conditions': ['type == "camp_combat_end"'], 'order_conditions': [{'node_used': 1, 'condition': 'start_id_A == end_id_B'}]}), ('e_3_not_0', {'node_conditions': ['type == "camp_combat_start"']})]}]
Step 1 initiated
Step 1 finished
Step 2 initiated
Step 2 finished
Step 3 initiated
Step 3 finished
Step 1 initiated
Step 1 finished
Step 2 initiated
Step 2 finished
Step 3 initiated
Step 3 finished


In [182]:
a_df

,user_id,event_e_0,event_e_1,event_e_2,event_e_4,event_e_3_not_0
9,144-1,298,389,406,408,NaN
10,144-1,298,399,407,408,NaN
804,292-2,239,283,330,370,NaN
805,292-2,239,284,331,370,NaN
806,292-2,239,294,332,370,NaN
807,292-2,239,295,333,370,NaN
808,292-2,239,302,334,370,NaN
809,292-2,239,303,335,370,NaN
810,292-2,239,310,336,370,NaN
811,292-2,239,311,337,370,NaN
